In [11]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time # for sleep time
import csv
import re
import functools
import datetime #for csv file title

driver = webdriver.Chrome(r'C:\Users\priya\chromedriver.exe')

# Tell Selenium to get the URL you're interested in.
driver.get('https://www.hotels.com/search.do?resolved-location=NEIGHBORHOOD%3A1535616%3AUNKNOWN%3AUNKNOWN&destination-id=1535616&q-destination=Manhattan,%20New%20York,%20New%20York,%20United%20States%20of%20America&q-rooms=1&q-room-0-adults=3&q-room-0-children=0')

def reduce_concat(x, sep=""):
    return functools.reduce(lambda x, y: str(x) + sep + str(y), x)

def paste(*lists, sep=" ", collapse=None):
    result = map(lambda x: reduce_concat(x, sep=sep), zip(*lists))
    if collapse is not None:
        return reduce_concat(result, sep=collapse)
    return list(result)

def scroll_bar():
    #scroll to end of page so that all elements are visible
    calm = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//html"))) #if elements aren't available it will return a timeout error and alert user that script must be altered
    time.sleep(3)
    lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match=False

    while(match==False):
        lastCount = lenOfPage
        time.sleep(3)
        lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        if lastCount==lenOfPage:
            match=True

    scroll = driver.find_element_by_xpath('//html')
    time.sleep(2)
    scroll.send_keys(Keys.HOME)
    time.sleep(2)

calm = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//html"))) #if elements aren't available it will return a timeout error and alert user that script must be altered
time.sleep(3)
lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
print(lenOfPage)
wait_event = WebDriverWait(driver, 10)
events = wait_event.until(EC.presence_of_all_elements_located((By.XPATH,'//div[@class="description resp-module"]')))

hotel_all_links = []
newPos = 0
while (newPos <= lenOfPage):
    print('Number of events:', len(events))
    print('position pre-loop:', newPos) #for testing
    scroll_bar()
    time.sleep(3)

    offprep = paste(["var offprep = window.screen.height * "], [(len(events))/5], [";return offprep"], sep='') #7 fit on a page, giving myself room for error
    offprep = ''.join(offprep)
    offset = driver.execute_script(offprep)
    newPos = newPos + offset
    scrollerArg = paste(["window.scrollTo(0, "], [newPos], [");"], sep = '')
    scrollerArg = ''.join(scrollerArg)
    time.sleep(20)
    scroller = driver.execute_script(scrollerArg)
    events = wait_event.until(EC.presence_of_all_elements_located((By.XPATH,'//div[@class="description resp-module"]')))

    print('Number of events:', len(events)) #testing
    print('current position:', newPos) #testing

# Get list of links to hotels and append   
time.sleep(40)
hotel_list = (driver.find_elements_by_xpath('//div[@class="description resp-module"]/figure[@class="image"]/a'))
hotel_all_links = [hotel.get_attribute('href') for hotel in hotel_list ] 

print(len(hotel_all_links))
print('+'*50)
print(hotel_all_links)
time.sleep(4)


time.sleep(1.5)

#Open CSV Writer
now = datetime.datetime.now() #for csv file title
filename = paste(['HotelsManh'], [now.year], ['-'], [now.month], ['-'], [now.day],['.csv'], sep='')
filename = ''.join(filename)
csv_file = open(filename, 'w', encoding='utf-8',newline = '')
writer = csv.writer(csv_file)


#enter the url of hotel after gathering all links and get the variables


for url in hotel_all_links:

    # Initialize an empty dictionary for each review
    #dictionary to store hotel data
    hotel_dict = {}
    try:
       
        driver.get(url)
       
        print('+'*50)
        print(url)
    except:
        continue
    try:
        time.sleep(1.5)
        try:
            hotel_title = driver.find_element_by_xpath('//div[@class="vcard"]/h1').text
        except Exception as e :
            print("issue in hotel title")
            print(e)
            hotel_title = None

        try:
            hotel_type =  driver.find_element_by_xpath('//div[@class="widget-query-group widget-query-destination"]/input').get_attribute('data-as-type')
        except:
            print("issue in hotel type")
            hotel_type = None

        try:

            hotel_price = driver.find_element_by_xpath('//span[@class="current-price"]').text
        except:
            
            try:
                hotel_price = driver.find_element_by_xpath('//div[@class="price"]/span[@class="current-price has-old-price"]').text
                
            except:
                print("issue in hotel_price")    
                hotel_price = None

        try:
            hotel_star = driver.find_element_by_xpath('//span[@class="star-rating-text star-rating-text-strong widget-star-rating-overlay"]').text

        except:
            try:
                hotel_star = driver.find_element_by_xpath('//span[@class="star-rating-text widget-star-rating-overlay"]').text

            except:
                print("issue in hotel star")
                hotel_star = None

        try:
            no_of_ratings = driver.find_element_by_xpath('//div/a[@class="total-reviews trust-you"]').text

        except Exception as e :
            print("issue in no_of_ratings")
            print(e)
            no_of_ratings = None  

        time.sleep(2)        
        try:
            coord_details = driver.find_elements_by_xpath('//span[@class="hotel-coordinates"]/meta')

            lat_long = [detail.get_attribute('content') for detail in coord_details ]

        except:
            print("Issue in lat_long")
            lat_long = None

        time.sleep(2.5)
        try:
            rating = driver.find_element_by_xpath('//div/span[@class="rating"]').text

        except:
            print("issue in rating")
            rating = None
            
        time.sleep(1)
        try:
            occupancy = driver.find_element_by_xpath('//div/span[@class="search-params"]').text

        except:
            print("issue in occupancy")
            occupancy = None
            
        hotel_dict['hotel_title'] = hotel_title
        hotel_dict['hotel_type'] = hotel_type
        hotel_dict['hotel_price'] = hotel_price
        hotel_dict['hotel_star'] = hotel_star
        hotel_dict['no_of_ratings'] = no_of_ratings
        hotel_dict['rating'] = rating
        hotel_dict['occupancy'] = occupancy
        hotel_dict['lat_long'] = lat_long
        
        time.sleep(2)
        print("writing")
        writer.writerow(hotel_dict.values())


        print(hotel_title)
        print(hotel_type)
        print(hotel_price)
        print(hotel_star)
        print(no_of_ratings)
        print(rating)
        print(occupancy)
        print(lat_long)
        
   
    
    except Exception as e:
        print(e)
        continue
        
csv_file.close()
driver.close()


6437
Number of events: 8
position pre-loop: 0
Number of events: 148
current position: 1382.4
Number of events: 148
position pre-loop: 1382.4
Number of events: 268
current position: 26956.800000000003
478
++++++++++++++++++++++++++++++++++++++++++++++++++
['https://www.hotels.com/travelads/trackredirect.html?trackingUrl=H4sIAAAAAAAAAD1Ux47jWAz8m75opiVZuQFjoZwsK1rBl4WildNTsr5-vTvAXkhWESQPVWC5LCP4geFljre8jTPw3QxZVn2nQwd7_3FsBtx83qo0hzccVoYlb8H_Lb6t0uavz3DaVP1LiJf4ynfj7ykS5GMPtU4QXbT4lL6aPHkmxpIF8OY4guIpJYYxJjTalnOiHCr-EnwX7dvzVrnwPV7rRN9kGImNwFoJZMuacZ9s7TjWOu8pvOv6uva4G-2xm_ju3vrDfx6uTg3L8B5WkntnL2KEoDsCHXSEJrxGUQ9awSl73V0k0n0Pf7U2XnB6RCjPS2A_xoZuUik8M255wqd_S49YC8DRMSn9NB7T2QZCRWChPmdFxbs16dtDTOr1fVjbBTMR_UayEsaRKh2pL2NnGxobH8NdC9-AV48I2mly4sv5EhET9n6qxmVN7Evqaw91A_eIyvib14SOy7Nooc8ltEGoj_MrwjTaWyUlskG65LVfosjyrdNHYA7GCsXQVVwZJeOo6OrhZ-YtwxzS5UzAn_AmF07ne5YZJrLghAgJUSYQWKqE5KXYw-0whjNxGbGIWtljZ4ZM6p5v4x2xk9NJjlfpkihduWR2mUBUFyPN9tra8sxxr1115E2pI0aSA4QsBrpnGkdP4Jt9sD6ILg02kSKdWlNHoAFEWzc

++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/travelads/trackredirect.html?trackingUrl=H4sIAAAAAAAAAD1Ux47jWAz8m75opiVZuQFjoZwsK1rBl4WildNTsr5-vTvAXkhWESQPVWC5LCP4geFljre8jTPw3QxZVn2nQwd7_3FsBtx83qo0hzccVoYlb8H_Lb6t0uavz3DaVP1LiJf4ynfj7ykS5GMPtU4QXbT4lL6aPHkmxpIF8OY4guIpJYYxJjTalnOiHCr-EnwX7dvzVrnwPV7rRN9kGImNwFoJZMuacZ9s7TjWOu8pvOv6uva4G-2xm_ju3vrDfx6uTg3L8B5WkntnL2KEoDsCHXSEJrxGUQ9awSl73V0k0n0Pf7U2XnB6RCjPS2A_xoZuUik8M255wqd_S49YC8DRMSn9NB7T2QZCRWChPmdFxbs16dtDTOr1fVjbBTMR_UayEsaRKh2pL2NnGxobH8NdC9-AV48I2mly4sv5EhET9n6qxmVN7Evqaw91A_eIyvib14SOy7Nooc8ltEGoj_MrwjTaWyUlskG65LVfosjyrdNHYA7GCsXQVVwZJeOo6OrhZ-YtwxzS5UzAn_AmF07ne5YZJrLghAgJUSYQWKqE5KXYw-0whjNxGbGIWtljZ4ZM6p5v4x2xk9NJjlfpkihduWR2mUBUFyPN9tra8sxxr1115E2pI0aSA4QsBrpnGkdP4Jt9sD6ILg02kSKdWlNHoAFEWzcaFpCqWfUCZmhZzGYIM4eLhdEw02CiMgylBpdNrxaOCQuTuGeTf6dPC2Y6wnC0c8GijkKlqOeU5inxXBgl_ho9mxGfmFJcxAvjVA_jrFyV5UnGqR8qosTvvTdZV5Iis5F9GxbIcgHrWouvS4QrLYGzouUvooDSPebpQpKdmnJ71qqSeYuR17auLFbNn0lrzIBIeULSHYymfYwcR

++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho485393/?pa=10&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
writing
Residence Inn Marriott New York Downtown Manhattan/WTC Area, New York
HOTEL
$148
3.5-star
See all 623 Hotels.com reviews
9.0
1 room, 3 adults
['40.709657', '-74.010093']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho198683/?pa=11&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
writing
The Sherry Netherland, New York
HOTEL
$459
4.5-star
See all 163 Hotels.com reviews
9.6
1 room, 3 adults
['40.76453', '-73.97285']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho133146/?pa=12&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
writing
The Franklin Hotel, New York
HOTEL
$207
3.5-star
See all 544 Hotels.com reviews
6.8
1 room, 3 adults
['40.779886', '-73.954404']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho540

++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho495029696/?pa=29&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
writing
Four Seasons Hotel New York Downtown, New York
HOTEL
$555
5-star
See all 171 Hotels.com reviews
9.6
1 room, 3 adults
['40.712544', '-74.009306']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho644566560/?pa=30&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
issue in hotel_price
issue in no_of_ratings
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div/a[@class="total-reviews trust-you"]"}
  (Session info: chrome=73.0.3683.103)
  (Driver info: chromedriver=2.46.628402 (536cd7adbad73a3783fdc2cab92ab2ba7ec361e1),platform=Windows NT 10.0.17134 x86_64)

writing
45th Street #226384 - 1 Br Condo, New York
HOTEL
None
3.5-star
None
Fabulous8.8
1 room, 3 adults
['40.75269', '-73.97276']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.ho

writing
NoMo SoHo, New York
HOTEL
$148
4.5-star
See all 885 Hotels.com reviews
8.4
1 room, 3 adults
['40.719921', '-74.000304']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho231553/?pa=51&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
writing
The Pierre, A Taj Hotel, New York, New York
HOTEL
$595
5-star
See all 267 Hotels.com reviews
9.4
1 room, 3 adults
['40.765341', '-73.972221']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho462980/?pa=52&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
issue in hotel_price
issue in no_of_ratings
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div/a[@class="total-reviews trust-you"]"}
  (Session info: chrome=73.0.3683.103)
  (Driver info: chromedriver=2.46.628402 (536cd7adbad73a3783fdc2cab92ab2ba7ec361e1),platform=Windows NT 10.0.17134 x86_64)

writing
onefinestay - Upper West Side private homes, New York
HOTEL
None
4.5-

...

writing
Perfect Retreat in East Village 2BR, New York
HOTEL
None
3-star
None
Fabulous8.8
1 room, 3 adults
['40.73', '-73.982']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho630004/?pa=398&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
issue in hotel_price
issue in no_of_ratings
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div/a[@class="total-reviews trust-you"]"}
  (Session info: chrome=73.0.3683.103)
  (Driver info: chromedriver=2.46.628402 (536cd7adbad73a3783fdc2cab92ab2ba7ec361e1),platform=Windows NT 10.0.17134 x86_64)

writing
East Village Bungalow 3BR, New York
HOTEL
None
3-star
None
Fabulous8.8
1 room, 3 adults
['40.73', '-73.982']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho630007/?pa=399&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
issue in hotel_price
issue in no_of_ratings
Message: no such element: Unable to locate element: {"method":"xp

writing
Serene Homes E25, New York
HOTEL
None
3-star
None
Fabulous8.8
1 room, 3 adults
['40.740863', '-73.983743']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho867133152/?pa=416&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
issue in hotel_price
issue in no_of_ratings
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div/a[@class="total-reviews trust-you"]"}
  (Session info: chrome=73.0.3683.103)
  (Driver info: chromedriver=2.46.628402 (536cd7adbad73a3783fdc2cab92ab2ba7ec361e1),platform=Windows NT 10.0.17134 x86_64)

writing
Lord & Moris - Times Square Hotel, New York
HOTEL
None
3-star
None
7.4
1 room, 3 adults
['40.75569', '-73.99159']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho568670/?pa=417&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
issue in hotel_price
issue in no_of_ratings
Message: no such element: Unable to locate element: {"method":"xpath"

writing
405 East 61st Street #5C 149 3 Bedrooms 1 Bathroom Apts, New York
HOTEL
None
None
None
Fabulous8.8
1 room, 3 adults
['40.760801', '-73.960446']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho1163565088/?pa=433&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
issue in hotel_price
issue in hotel star
issue in no_of_ratings
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div/a[@class="total-reviews trust-you"]"}
  (Session info: chrome=73.0.3683.103)
  (Driver info: chromedriver=2.46.628402 (536cd7adbad73a3783fdc2cab92ab2ba7ec361e1),platform=Windows NT 10.0.17134 x86_64)

writing
104 West 83rd Street #3B 195 1 Bedroom 1 Bathroom Apts, New York
HOTEL
None
None
None
Fabulous8.8
1 room, 3 adults
['40.784577', '-73.974227']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho578930/?pa=434&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
writing
3 West Club, New Y

writing
Hotel Indigo Lower East Side New York, New York
HOTEL
$178
4.5-star
See all 771 Hotels.com reviews
8.8
1 room, 3 adults
['40.721736', '-73.987497']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho452020/?pa=452&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
writing
Hotel 32 32, New York
HOTEL
$124
4-star
See all 1,330 Hotels.com reviews
8.0
1 room, 3 adults
['40.746207', '-73.983351']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho196022/?pa=453&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
writing
Hudson River Hotel, New York
HOTEL
$109
2.5-star
See all 315 Hotels.com reviews
7.4
1 room, 3 adults
['40.755483', '-73.99733']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho480689/?pa=454&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
issue in hotel_price
issue in no_of_ratings
Message: no such element: Unable to locate element: {"method":"

writing
The Wagner Hotel, New York
HOTEL
$223
5-star
None
9.0
1 room, 3 adults
['40.70552', '-74.01744']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho237052/?pa=471&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
writing
The Marmara Manhattan, New York
HOTEL
$286
3.5-star
See all 335 Hotels.com reviews
8.4
1 room, 3 adults
['40.782847', '-73.947762']
++++++++++++++++++++++++++++++++++++++++++++++++++
https://www.hotels.com/ho669000160/?pa=472&tab=description&ZSX=0&SYE=3&q-room-0-children=0&q-room-0-adults=3
issue in no_of_ratings
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div/a[@class="total-reviews trust-you"]"}
  (Session info: chrome=73.0.3683.103)
  (Driver info: chromedriver=2.46.628402 (536cd7adbad73a3783fdc2cab92ab2ba7ec361e1),platform=Windows NT 10.0.17134 x86_64)

writing
Freehand New York, New York
HOTEL
$129
4-star
None
8.8
1 room, 3 adults
['40.73973', '-73.98442']
++++++++++++++++++++